# Mount Google Drive and change directory to the location of your MD simulation files


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/MD_Simulation_Results/unbound_dsba")

In [ ]:
!pwd

/content/drive/MyDrive/MD_Simulation_Results/aloeresin_a_dsba_complex


# Install Gromacs

In [ ]:
! apt-get update && apt-get install -y gromacs

In [ ]:
!gmx --version

# Clean the protein

In [ ]:
grep -v HOH dsbs.pdb > dsba_clean.pdb

# Generate a GROMACS topology for the protein


In [ ]:
!gmx pdb2gmx -f dsba_clean.pdb -ff charmm27 -water tip3p -ignh -o dsba_processed.gro -nochargegrp

# Add solvent and ions to the complex

Define a box and solvate the system. Also add ions (using the `ions.mdp script`) to neutralize the system


In [ ]:
!gmx editconf -f dsba_processed.gro -o boxed.gro -c -d 1.2 -bt octahedron

In [ ]:
!gmx solvate -cs -cp boxed.gro -o solvated.gro -p topol.top

In [ ]:
!gmx grompp -f ions.mdp -c solvated.gro -p topol.top -o ions.tpr

In [ ]:
!gmx genion -s ions.tpr -o solvated_ions.gro -p topol.top -pname NA -nname CL -neutral

# Energy minimize the system using the `em.mdp script`

In [ ]:
!gmx grompp -f minim.mdp -c solvated_ions.gro -p topol.top -o em.tpr

In [ ]:
!gmx mdrun -v -deffnm em

Let's do a bit of analysis. The em.edr file contains all of the energy terms that GROMACS collects during EM. You can analyze any .edr file using the GROMACS energy module. At the prompt, type "12 0" to select Potential (12); zero (0) terminates input.

In [ ]:
!gmx energy -f em.edr -o potential.xvg

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load data from the xvg file
data = np.loadtxt('potential.xvg', comments=['#', '@'])

# Extract columns (adjust column indices as needed)
time = data[:, 0]
energy = data[:, 1]

# Plot the data
fig = plt.figure(figsize=(10,5))
plt.plot(time, energy, label='Energy')
plt.xlabel('Time (ps)')
plt.ylabel('Potential energy (kcal/mol)')
plt.legend()
plt.show()

# Perform equilibration
Equilibration is often conducted in two phases. The first phase is conducted under an NVT ensemble (constant Number of particles, Volume, and Temperature). This ensemble is also referred to as "isothermal-isochoric" or "canonical." The timeframe for such a procedure is dependent upon the contents of the system, but in NVT, the temperature of the system should reach a plateau at the desired value. If the temperature has not yet stabilized, additional time will be required. Typically, 50-100 ps should suffice, and we will conduct a 100-ps NVT equilibration for this exercise. Depending on your machine, this may take a while (just under an hour if run in parallel on 16 cores or so).

NVT equilibration, stabilized the temperature of the system. Prior to data collection, we must also stabilize the pressure (and thus also the density) of the system. Equilibration of pressure is conducted under an NPT ensemble, wherein the Number of particles, Pressure, and Temperature are all constant. The ensemble is also called the "isothermal-isobaric" ensemble, and most closely resembles experimental conditions.

**Now, run the NVT equilibration using the `nvt.mdp script`**

In [ ]:
!gmx grompp -f nvt.mdp -c em.gro -r em.gro -p topol.top -n index.ndx -o nvt.tpr
!gmx mdrun -v -deffnm nvt

Let's analyze the temperature progression, again using energy:

In [ ]:
!gmx energy -f nvt.edr -o temperature.xvg

Type "17 0" at the prompt to select the temperature of the system and exit. The resulting plot should look something like the following:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data from the xvg file
data = np.loadtxt('temperature.xvg', comments=['#', '@'])

# Extract columns (adjust column indices as needed)
time = data[:, 0]
temperature = data[:, 1]

# Calculate running average (adjust window size as needed)
window_size = 10
running_average = np.convolve(temperature, np.ones(window_size)/window_size,
                              mode='valid')

# Plot the data and running average
fig = plt.figure(figsize=(10,5))
plt.plot(time, temperature, label='Temperature')
plt.plot(time[:len(running_average)], running_average,
         label=f'Running Average', linestyle='--', color='red')
plt.yticks(np.arange(292.0, 310, step=2))
plt.xlabel('Time (ps)')
plt.ylabel('Temperature (K)')
plt.legend()

# Set x-axis and y-axis limits to start from 0
plt.xlim(left=0)
# plt.ylim(bottom=-350)

plt.show()

**After the NVT equilibration, run the NPT equilibration using the `npt.mdp script`**

In [ ]:
!gmx grompp -f npt.mdp -c nvt.gro -t nvt.cpt -r nvt.gro -p topol.top -n index.ndx -o npt.tpr
!gmx mdrun -v -deffnm npt

Let's analyze the pressure progression, again using energy. Type "18 0" at the prompt to select the pressure of the system and exit. The resulting plot should look something like the following:

In [ ]:
!gmx energy -f npt.edr -o pressure.xvg

The resulting plot should look something like the following:

In [ ]:
# Load data from the xvg file
data = np.loadtxt('pressure.xvg', comments=['#', '@'])

# Extract columns (adjust column indices as needed)
time = data[:, 0]
pressure = data[:, 1]

# Calculate running average (adjust window size as needed)
window_size = 10
running_average = np.convolve(pressure, np.ones(window_size)/window_size,
                              mode='valid')

# Plot the data and running average
fig = plt.figure(figsize=(10,5))
plt.plot(time, pressure, label='Pressure')
plt.plot(time[:len(running_average)], running_average,
         label=f'Running Average', linestyle='--', color='red')
plt.yticks(np.arange(-400.0, 450, step=200))
plt.xlabel('Time (ps)')
plt.ylabel('Pressure (bar)')
plt.legend()

# Set x-axis and y-axis limits to start from 0
plt.xlim(left=0)
# plt.ylim(bottom=-350)

plt.show()

Let's take a look at density as well, this time using energy and entering "24 0" at the prompt.

In [ ]:
# !gmx energy -f npt.edr -o density.xvg

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data from the xvg file
data = np.loadtxt('density.xvg', comments=['#', '@'])

# Extract columns (adjust column indices as needed)
time = data[:, 0]
density = data[:, 1]

# Calculate running average (adjust window size as needed)
window_size = 10
running_average = np.convolve(density, np.ones(window_size)/window_size,
                              mode='valid')

# Plot the data and running average
fig = plt.figure(figsize=(10,5))
plt.plot(time, density, label='Density')
plt.plot(time[:len(running_average)], running_average,
         label=f'Running Average', linestyle='--', color='red')
plt.yticks(np.arange(990.0, 1004, step=2))
plt.xlabel('Time (ps)')
plt.ylabel('Density (kg/m^-3)')
plt.legend()

# Set x-axis limits to start from 0
plt.xlim(left=0)

plt.show()

# Production Stage
Upon completion of the two equilibration phases, the system is now well-equilibrated at the desired temperature and pressure. We are now ready to release the position restraints and run production MD for data collection. The process is just like we have seen before, as we will make use of the checkpoint file (which in this case now contains preserve pressure coupling information) to grompp. We will run a 10-ns MD simulation using the md.mdp script.

In [ ]:
!gmx grompp -f md.mdp -c npt.gro -t npt.cpt -p topol.top -o md_0_10.tpr

In [ ]:
# Check for GPU availability

import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())

In [ ]:
!export GMX_GPU_ID = torch.cuda.current_device()
!gmx mdrun -v -cpi md_0_10.cpt -deffnm md_0_10

# !gmx mdrun -v -cpi md_0_10.cpt -deffnm md_0_10 -nb gpu

# Analysis of the results

Now that we have simulated our protein, we should run some analysis on the system. What types of data are important? This is an important question to ask before running the simulation, so you should have some ideas about the types of data you will want to collect in your own systems. For this tutorial, a few basic tools will be introduced.

The first is trjconv, which is used as a post-processing tool to strip out coordinates, correct for periodicity, or manually alter the trajectory (time units, frame frequency, etc). For this exercise, we will use trjconv to account for any periodicity in the system. The protein will diffuse through the unit cell, and may appear "broken" or may "jump" across to the other side of the box. To account for such behaviors, issue the following:

In [ ]:
!gmx trjconv -s md_0_1.tpr -f md_0_1.xtc -o md_0_1_noPBC.xtc -pbc mol -center

Select 1 ("Protein") as the group to be centered and 0 ("System") for output. We will conduct all our analyses on this "corrected" trajectory.

**Root-Mean-Square-Deviation (RMSD)**

GROMACS has a built-in utility for RMSD calculations called rms. To use rms, issue this command:

In [ ]:
!gmx rms -s md_0_1.tpr -f md_0_1_noPBC.xtc -o rmsd.xvg -tu ns

Choose 4 ("Backbone") for both the least-squares fit and the group for RMSD calculation. The -tu flag will output the results in terms of ns, even though the trajectory was written in ps. This is done for clarity of the output (especially if you have a long simulation - 1e+05 ps does not look as nice as 100 ns). The output plot will show the RMSD relative to the structure present in the minimized, equilibrated system:

If we wish to calculate RMSD relative to the crystal structure, we could issue the following:

In [ ]:
!gmx rms -s em.tpr -f md_0_1_noPBC.xtc -o rmsd_xtal.xvg -tu ns

**Radius of Gyration (Rg)**

The radius of gyration of a protein is a measure of its compactness. If a protein is stably folded, it will likely maintain a relatively steady value of Rg. If a protein unfolds, its Rg will change over time. Let's analyze the radius of gyration for lysozyme in our simulation:

In [ ]:
!gmx gyrate -s md_0_1.tpr -f md_0_1_noPBC.xtc -o gyrate.xvg

Choose group 1 (Protein) for analysis.

**Root-Mean-Square Fluctuation (RMSF)**


In [ ]:
!gmx rmsf -s md_0_1.tpr -f md_0_1_noPBC.xtc -o rmsf.xvg

# Further analysis

Performing a detailed analysis of molecular dynamics (MD) simulations involves several steps. Here's a general guide on how you can calculate and visualize various properties using GROMACS and generate a movie for the trajectory:

1. **Calculate RMSD (Root Mean Square Deviation):**  

`gmx rms -s md_0_1.tpr -f md_0_1_noPBC.xtc -o rmsd.xvg`

This will calculate the RMSD and save the data in rmsd.xvg.

2. **Calculate RMSF (Root Mean Square Fluctuation):**

`gmx rmsf -s md_0_1.tpr -f md_0_1_noPBC.xtc -o rmsf.xvg`

3. **Calculate Rg (Radius of Gyration):**

`gmx gyrate -s md_0_1.tpr -f md_0_1_noPBC.xtc -o gyrate.xvg`

4. **Calculate Interaction Energy:**

`gmx energy -f md_0_1.edr -s md_0_1.tpr -o potential.xvg`

5. **Hydrogen Bond Analysis:**

`gmx hbond -s md_0_1.tpr -f md_0_1_noPBC.xtc -num hbond.xvg`

6. **Plotting Graphs:**

You can use tools like `matplotlib (in python)`, `xmgrace`, `gnuplot`, `Excel`, etc. to plot the data from the generated .xvg files.

7. **Generate a Movie:**

`gmx trjconv -s md_0_1.tpr -f md_0_1_noPBC.xtc -o trajout.xtc -pbc mol`

Then, use a visualization tool like VMD or PyMOL to create a trajectory movie:

**VMD:**

`vmd -gro md_0_1.gro -xtc trajout.xtc`

In VMD, use the "Animate" tab to create a movie.

**PyMOL:**

`pymol md_0_1.gro trajout.xtc`

In PyMOL, use the "Movie" panel to save the trajectory as a movie.

Make sure to adjust filenames and paths according to your specific setup.

Remember to check GROMACS documentation and user manuals for the most up-to-date information and options.